In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json
import os

import pandas as pd
import requests
from dotenv import load_dotenv
from IPython.display import clear_output

In [3]:
load_dotenv()

API_HOST = os.getenv("API_HOST")
API_PORT = os.getenv("API_PORT")
REDIS_HOST = os.getenv("REDIS_HOST")
REDIS_PORT = os.getenv("REDIS_PORT")

# SkillGPT API Request Demo

## Summarize

In [4]:
prompt = """
### Human: I want you to act as a human resource expert and summarize the top five skills required by following user profile:
----
Working Experience
1. Front End Developer, ABC Tech Solutions (May 2020 - July 2022)
    * Developed and maintained user-friendly, responsive web applications using HTML, CSS, and JavaScript for a diverse clientele, resulting in a 30% increase in customer satisfaction.
    * Collaborated closely with UX/UI designers and back-end developers to implement seamless integration of design and functionality, optimizing load times by 20%.
2. Junior Front End Developer, XYZ Web Agency (January 2018 - April 2020)
    * Assisted in the creation of mobile-first web designs, adhering to accessibility standards and ensuring cross-browser compatibility, which led to a 25% increase in mobile traffic.
    * Actively participated in agile development sprints, contributing to efficient project management and improving the overall quality of deliverables.
3. Intern, Front End Development, LMN Digital (June 2017 - December 2017)
    * Gained hands-on experience in modern web development frameworks, including React and Angular, through the creation of interactive web components.
    * Assisted in troubleshooting and resolving front-end issues, refining code quality, and optimizing website performance.
----
### Assistant:          
"""

In [5]:
sep = "###"
worker_addr = f"http://{API_HOST}:{API_PORT}"
headers = {"User-Agent": "SkillGPT Client"}
pload = {
    "model": "vicuna-13b",
    "prompt": prompt,
    "max_new_tokens": 500,
    "temperature": 0.7,
    "stop": sep,
}
response = requests.post(worker_addr + "/generate_stream", headers=headers,
        json=pload, stream=True)

for chunk in response.iter_lines(chunk_size=8192, decode_unicode=False, delimiter=b"\0"):
    if chunk:
        data = json.loads(chunk.decode("utf-8"))
        output = data["text"].split(sep)[-1]
        clear_output(wait=True)
        print(f"\r{output}", end="")

 Assistant:          
1. Proficiency in Front-End Development Tools: HTML, CSS, and JavaScript
2. Experience with User Experience (UX) and User Interface (UI) Design
3. Collaboration and Communication Skills
4. Knowledge of Development Frameworks (e.g. React, Angular)
5. Familiarity with Agile Development Methodologies


## Label text

In [6]:
def format_esco_concepts(esco_concepts, esco_index):
    df_res = pd.DataFrame.from_records([json.loads(concept_str) for concept_str in esco_concepts])
    df_res["rank"] = range(1, 1+len(df_res))
    return df_res[esco_fields[esco_index]]

esco_fields = { 
    "occupations": ["rank", "preferredLabel", "conceptType", "code", "altLabels", "description", "conceptUri"],
    "skillGroups": ["rank", "preferredLabel", "conceptType", "code", "altLabels", "description", "conceptUri"],
    "skills": ["rank", "preferredLabel", "conceptType", "skillType", "altLabels", "description", "conceptUri"]
}

In [11]:
prompt = """
1. Proficiency in Front-End Development Tools: HTML, CSS, and JavaScript
2. Experience with User Experience (UX) and User Interface (UI) Design
3. Collaboration and Communication Skills
4. Knowledge of Development Frameworks (e.g. React, Angular)
5. Familiarity with Agile Development Methodologies
"""
esco_index = "occupations"
sep = "###"
worker_addr = f"http://{API_HOST}:{API_PORT}"
headers = {"User-Agent": "SkillGPT Client"}
pload = {
    "model": "vicuna-13b",
    "prompt": prompt,
    "esco_index": esco_index,
    "redis_host": REDIS_HOST,
    "num_relevant": 10,
}
response = requests.post(worker_addr + "/label_text", headers=headers,
        json=pload, stream=False)

format_esco_concepts(json.loads(response.content)["labels"], esco_index)

,rank,preferredLabel,conceptType,code,altLabels,description,conceptUri
0,1,user interface developer,Occupation,2512.5,user interface developers\nUI developers\nfron...,"User interface developers implement, code, doc...",http://data.europa.eu/esco/occupation/866c7813...
1,2,user interface designer,Occupation,2513.3,user interface designer\nUI designer\naccessib...,User interface designers are in charge of desi...,http://data.europa.eu/esco/occupation/96e20037...
2,3,user interface developer,Occupation,2512.5,user interface designer\nUI designer\ninteract...,"User interface developers implementeren, coder...",http://data.europa.eu/esco/occupation/866c7813...
3,4,web developer,Occupation,2513.5,web developers\nweb engineer\nweb application ...,"Web developers develop, implement and document...",http://data.europa.eu/esco/occupation/c40a2919...
4,5,ICT application developer,Occupation,2514.2,application software developers\nICT applicati...,ICT application developers implement the ICT (...,http://data.europa.eu/esco/occupation/bd272aee...
5,6,blockchain developer,Occupation,2512.4.1,decentralized ledger technology specialist\nDL...,Blockchain developers implement or program blo...,http://data.europa.eu/esco/occupation/24135b84...
6,7,database developer,Occupation,2521.3,database programmer\ndatabase development engi...,"Database developers program, implement and coo...",http://data.europa.eu/esco/occupation/b11e1742...
7,8,mobile application developer,Occupation,2514.2.2,app coder\napp programmer\ncell phone app deve...,Mobile application developers implement applic...,http://data.europa.eu/esco/occupation/2ed56c3f...
8,9,3D modeller,Occupation,2166.1,3D modellers\n3D texturing artist\n3D modeler\...,"3D modellers design 3D models of objects, virt...",http://data.europa.eu/esco/occupation/bab5fa79...
9,10,software developer,Occupation,2512.4,software specialist\nsoftware developers\nprog...,Software developers implement or program all k...,http://data.europa.eu/esco/occupation/f2b15a0e...


# Initialize ESCO embeddings

#### Note: this will wipe the entire redis and ingest new embeddings

In [ ]:
worker_addr = f"http://{REDIS_HOST}:{REDIS_PORT}"
headers = {"User-Agent": "SkillGpt Client"}
pload = {
    "model": "vicuna-13b",
    "redis_host": REDIS_HOST,
}
response = requests.post(worker_addr + "/init_esco_embedding", headers=headers,
        json=pload)

print(response.content)